In [1]:
import cv2
import easyocr
from IPython.display import Image, display
import numpy as np
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import re

Ultralytics YOLOv8.0.117 🚀 Python-3.9.13 torch-2.0.1+cu117 CPU
Setup complete ✅ (4 CPUs, 3.7 GB RAM, 90.0/233.2 GB disk)


In [2]:
# !pip install ultralytics

In [3]:
# Load a pretrained model
model = YOLO('best.pt')
model.conf = 0.4  # Confidence threshold for detection


In [4]:
def preprocess_image(image):
    # Convert to grayscale
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Display the grayscale image
    display(Image(data=cv2.imencode('.jpg',grayscale_image)[1].tobytes()))
    # # Apply denoising filter
    # denoised_image = cv2.GaussianBlur(grayscale_image, (3, 3), 0)
    # Apply contrast enhancement
    # equalized_image = cv2.equalizeHist(grayscale_image)
    # Display the preprocessed image
    # display(Image(data=cv2.imencode('.jpg', equalized_image)[1].tobytes()))
    # Apply image sharpening
    sharpened_image = cv2.filter2D(grayscale_image, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
    # Display the preprocessed image
    display(Image(data=cv2.imencode('.jpg', sharpened_image)[1].tobytes()))
    return grayscale_image

In [5]:
def threshold(image):
    # Apply Otsu thresholding
    _, threshold_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Display the threshold image
    display(Image(data=cv2.imencode('.jpg',threshold_image)[1].tobytes()))
    return threshold_image

In [6]:
def ocrImage(image):
    # Perform OCR on the thresholded image
    recognized_plates=[]
    reader = easyocr.Reader(['en', 'ne'])
    result = reader.readtext(image, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ ', detail=0)
    recognized_plates = [''.join(result)]
    return recognized_plates


In [7]:
# reader = easyocr.Reader(['en', 'ne'])

In [20]:
def recognize_number_plate(video_path):
    best_plate = None
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Initialize variables to keep track of previous recognized plate
    prev_plate = None
    
    # Read frames from the video
    while True:
        ret, frame = video.read()
        
        # Check if a frame was successfully read
        if not ret:
            break
        
        # Perform object detection using YOLO
        detections = model(frame)
        
        # Extract bounding boxes and crop number plate regions
        number_plate_box = None
        for detection in detections[0].boxes.data:
            if detection[5] == 0:  
                number_plate_box = detection[:4]
                break
        
        # Crop the number plate region
        if number_plate_box is not None:
            x1, y1, x2, y2 = number_plate_box
            cropped_image = frame[int(y1):int(y2), int(x1):int(x2)]
            
            # Perform OCR on the number plate image
            result = reader.readtext(cropped_image, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ ', detail=0)
            
            # Extract the text from the OCR result
            recognized_plate = ''.join(result)
            
            
            # Check if the recognized plate is different from the previous plate
            if recognized_plate != prev_plate:
                if best_plate is None or len(recognized_plate) > len(best_plate):
#                best_frame = frame.copy()
                    best_plate = recognized_plate
                    # Print the recognized plate for the new vehicle
                    print("Number Plate:",best_plate)
                
                # Update the previous plate with the new plate
                prev_plate = recognized_plate
    
    # Release the video file
    video.release()


In [21]:
# Path to video file
video_path = 'car.mp4'
recognized_plates = recognize_number_plate(video_path)


0: 544x800 2 License-plates, 429.0ms
Speed: 11.0ms preprocess, 429.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)



Number Plate: R183 JF


0: 544x800 2 License-plates, 523.8ms
Speed: 9.1ms preprocess, 523.8ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 629.3ms
Speed: 14.4ms preprocess, 629.3ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 491.3ms
Speed: 6.8ms preprocess, 491.3ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 427.7ms
Speed: 10.2ms preprocess, 427.7ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 439.1ms
Speed: 7.0ms preprocess, 439.1ms inference, 1.2ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 417.6ms
Speed: 7.0ms preprocess, 417.6ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 410.7ms
Speed: 7.0ms preprocess, 410.7ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 420.2ms
Speed: 7.1ms

Number Plate: R 183 JF


0: 544x800 1 License-plate, 422.1ms
Speed: 6.7ms preprocess, 422.1ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 415.8ms
Speed: 6.4ms preprocess, 415.8ms inference, 1.5ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 520.5ms
Speed: 10.8ms preprocess, 520.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 511.9ms
Speed: 6.2ms preprocess, 511.9ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 410.8ms
Speed: 7.0ms preprocess, 410.8ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)



Number Plate: IR 183 JF


0: 544x800 2 License-plates, 419.5ms
Speed: 6.8ms preprocess, 419.5ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 429.2ms
Speed: 7.5ms preprocess, 429.2ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 433.4ms
Speed: 6.5ms preprocess, 433.4ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 417.1ms
Speed: 6.3ms preprocess, 417.1ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 497.0ms
Speed: 6.5ms preprocess, 497.0ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 454.3ms
Speed: 6.7ms preprocess, 454.3ms inference, 1.1ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 411.6ms
Speed: 7.5ms preprocess, 411.6ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 407.4ms
Speed: 7.0ms 


0: 544x800 1 License-plate, 414.6ms
Speed: 6.0ms preprocess, 414.6ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 528.6ms
Speed: 8.1ms preprocess, 528.6ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 510.9ms
Speed: 12.6ms preprocess, 510.9ms inference, 2.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 483.3ms
Speed: 8.2ms preprocess, 483.3ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 460.1ms
Speed: 9.5ms preprocess, 460.1ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 469.8ms
Speed: 6.5ms preprocess, 469.8ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 476.1ms
Speed: 6.3ms preprocess, 476.1ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 496.1ms
Speed: 6.5ms pr


0: 544x800 1 License-plate, 454.2ms
Speed: 6.8ms preprocess, 454.2ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 495.4ms
Speed: 6.2ms preprocess, 495.4ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 612.6ms
Speed: 6.1ms preprocess, 612.6ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 443.0ms
Speed: 8.1ms preprocess, 443.0ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 425.2ms
Speed: 6.6ms preprocess, 425.2ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 1 License-plate, 444.4ms
Speed: 6.4ms preprocess, 444.4ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 (no detections), 447.2ms
Speed: 7.4ms preprocess, 447.2ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 800)

0: 544x800 2 License-plates, 414.8ms
Speed: 8.5ms pre